# Day 02: Model Monitoring & Drift Detection

## Week 23: Production ML for Quantitative Finance

---

### Learning Objectives
- Understand why model monitoring is critical for trading systems
- Learn different types of drift: Data Drift, Concept Drift, Prediction Drift
- Implement statistical tests for drift detection (KS test, PSI, Chi-squared)
- Build automated alerting systems for production models
- Apply monitoring techniques to financial time series

### Why Model Monitoring Matters in Finance
In production trading systems, models can degrade silently due to:
- **Market regime changes** (bull → bear, low → high volatility)
- **Feature distribution shifts** (data drift)
- **Changing relationships** between features and returns (concept drift)
- **Data quality issues** (missing data, API failures)

A model that worked well in backtesting can become unprofitable without proper monitoring!

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Statistical testing
from scipy import stats
from scipy.stats import ks_2samp, chi2_contingency, entropy
from scipy.spatial.distance import jensenshannon

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Generate Synthetic Training and Production Data

We'll simulate a trading signal classification problem where:
- **Training Data**: Collected during a "normal" market regime
- **Production Data**: Collected after deployment, with intentional drift

### Types of Drift We'll Simulate:
1. **Data Drift**: Feature distributions shift (e.g., volatility regime change)
2. **Concept Drift**: The relationship between features and target changes
3. **Gradual Drift**: Slow changes over time
4. **Sudden Drift**: Abrupt market regime change

In [ ]:
def generate_financial_features(n_samples: int, 
                                   regime: str = 'normal',
                                   drift_factor: float = 0.0) -> pd.DataFrame:
    """
    Generate synthetic financial features for a trading signal classifier.
    
    Features:
    - momentum: Price momentum indicator
    - volatility: Rolling volatility
    - volume_ratio: Volume relative to average
    - rsi: Relative Strength Index (normalized)
    - spread: Bid-ask spread
    - market_cap_log: Log market cap category
    """
    
    if regime == 'normal':
        # Normal market conditions (training data)
        momentum = np.random.normal(0.0, 1.0, n_samples)
        volatility = np.abs(np.random.normal(0.15, 0.05, n_samples))
        volume_ratio = np.random.lognormal(0, 0.3, n_samples)
        rsi = np.random.beta(5, 5, n_samples)  # Centered around 0.5
        spread = np.abs(np.random.normal(0.001, 0.0003, n_samples))
        
    elif regime == 'high_volatility':
        # High volatility regime (data drift)
        momentum = np.random.normal(0.0, 1.5, n_samples)  # Higher variance
        volatility = np.abs(np.random.normal(0.25, 0.10, n_samples))  # Shifted mean
        volume_ratio = np.random.lognormal(0.3, 0.5, n_samples)  # Higher volume
        rsi = np.random.beta(3, 7, n_samples)  # Skewed towards oversold
        spread = np.abs(np.random.normal(0.002, 0.0008, n_samples))  # Wider spreads
        
    elif regime == 'trending':
        # Strong trending market
        momentum = np.random.normal(0.5 + drift_factor, 0.8, n_samples)  # Positive bias
        volatility = np.abs(np.random.normal(0.12, 0.03, n_samples))  # Lower vol
        volume_ratio = np.random.lognormal(0.2, 0.25, n_samples)
        rsi = np.random.beta(7, 3, n_samples)  # Skewed towards overbought
        spread = np.abs(np.random.normal(0.0008, 0.0002, n_samples))
        
    else:  # gradual drift
        # Gradual shift in distributions
        momentum = np.random.normal(drift_factor * 0.3, 1.0 + drift_factor * 0.2, n_samples)
        volatility = np.abs(np.random.normal(0.15 + drift_factor * 0.05, 0.05, n_samples))
        volume_ratio = np.random.lognormal(drift_factor * 0.1, 0.3, n_samples)
        rsi = np.random.beta(5 - drift_factor, 5 + drift_factor, n_samples)
        spread = np.abs(np.random.normal(0.001 + drift_factor * 0.0005, 0.0003, n_samples))
    
    # Market cap category (categorical feature)
    market_cap_log = np.random.choice([1, 2, 3, 4, 5], n_samples, 
                                       p=[0.1, 0.2, 0.3, 0.25, 0.15])
    
    return pd.DataFrame({
        'momentum': momentum,
        'volatility': volatility,
        'volume_ratio': volume_ratio,
        'rsi': rsi,
        'spread': spread,
        'market_cap': market_cap_log
    })


def generate_labels(features: pd.DataFrame, 
                    regime: str = 'normal',
                    noise_level: float = 0.1) -> np.ndarray:
    """
    Generate binary labels (1 = profitable trade signal, 0 = not profitable).
    
    The relationship between features and labels changes based on regime
    to simulate concept drift.
    """
    n_samples = len(features)
    
    if regime == 'normal':
        # Normal relationship: momentum and low volatility predict positive returns
        logits = (
            0.5 * features['momentum'] +
            -2.0 * features['volatility'] +
            0.3 * features['volume_ratio'] +
            1.0 * (features['rsi'] - 0.5) +
            -100 * features['spread']
        )
    elif regime == 'concept_drift':
        # CONCEPT DRIFT: Relationship changes!
        # Now high volatility and momentum reversal predict returns
        logits = (
            -0.3 * features['momentum'] +  # Momentum reversal now works
            1.5 * features['volatility'] +  # High vol is now good
            -0.2 * features['volume_ratio'] +
            -0.5 * (features['rsi'] - 0.5) +  # RSI relationship inverted
            50 * features['spread']
        )
    else:
        # Same as normal
        logits = (
            0.5 * features['momentum'] +
            -2.0 * features['volatility'] +
            0.3 * features['volume_ratio'] +
            1.0 * (features['rsi'] - 0.5) +
            -100 * features['spread']
        )
    
    # Convert to probabilities and add noise
    probs = 1 / (1 + np.exp(-logits))
    probs = np.clip(probs + np.random.normal(0, noise_level, n_samples), 0.01, 0.99)
    
    # Generate binary labels
    labels = (np.random.random(n_samples) < probs).astype(int)
    
    return labels

print("✅ Data generation functions created!")

In [ ]:
# Generate Training Data (Normal Market Regime)
print("=" * 60)
print("GENERATING DATASETS")
print("=" * 60)

n_train = 5000
n_production = 1000

# Training data - normal market conditions
X_train = generate_financial_features(n_train, regime='normal')
y_train = generate_labels(X_train, regime='normal')

print(f"\n📊 Training Data (Normal Regime):")
print(f"   Samples: {len(X_train)}")
print(f"   Class distribution: {np.bincount(y_train)}")
print(f"   Class 1 ratio: {y_train.mean():.2%}")

# Production Data - Different Scenarios
# Scenario 1: No Drift (baseline)
X_prod_no_drift = generate_financial_features(n_production, regime='normal')
y_prod_no_drift = generate_labels(X_prod_no_drift, regime='normal')

# Scenario 2: Data Drift (high volatility regime)
X_prod_data_drift = generate_financial_features(n_production, regime='high_volatility')
y_prod_data_drift = generate_labels(X_prod_data_drift, regime='normal')  # Same relationship

# Scenario 3: Concept Drift (relationship changes)
X_prod_concept_drift = generate_financial_features(n_production, regime='normal')
y_prod_concept_drift = generate_labels(X_prod_concept_drift, regime='concept_drift')

# Scenario 4: Both Data and Concept Drift
X_prod_both_drift = generate_financial_features(n_production, regime='high_volatility')
y_prod_both_drift = generate_labels(X_prod_both_drift, regime='concept_drift')

print(f"\n📊 Production Datasets Generated:")
print(f"   1. No Drift: {len(X_prod_no_drift)} samples")
print(f"   2. Data Drift (High Vol): {len(X_prod_data_drift)} samples")
print(f"   3. Concept Drift: {len(X_prod_concept_drift)} samples")
print(f"   4. Both Drifts: {len(X_prod_both_drift)} samples")

In [ ]:
# Visualize feature distributions: Training vs Production (Data Drift)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
features = ['momentum', 'volatility', 'volume_ratio', 'rsi', 'spread', 'market_cap']

for idx, feature in enumerate(features):
    ax = axes[idx // 3, idx % 3]
    
    # Training distribution
    ax.hist(X_train[feature], bins=50, alpha=0.5, label='Training', density=True, color='blue')
    # Production with data drift
    ax.hist(X_prod_data_drift[feature], bins=50, alpha=0.5, label='Production (Data Drift)', 
            density=True, color='red')
    
    ax.set_title(f'{feature.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.legend()
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')

plt.suptitle('Feature Distributions: Training vs Production (Data Drift Scenario)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n🔍 Notice how the distributions shifted in the Data Drift scenario:")
print("   - Volatility: Mean shifted from ~0.15 to ~0.25")
print("   - Spread: Wider spreads in production")
print("   - RSI: Skewed towards oversold conditions")

## 3. Train a Baseline Model

We'll train a Random Forest classifier to predict profitable trading signals. This model will serve as our baseline for monitoring.

In [ ]:
# Split training data for validation
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val)

# Train Random Forest model
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_scaled, y_train_split)

# Evaluate on validation set
y_val_pred = model.predict(X_val_scaled)
y_val_proba = model.predict_proba(X_val_scaled)[:, 1]

print("=" * 60)
print("BASELINE MODEL PERFORMANCE (Validation Set)")
print("=" * 60)
print(f"\nAccuracy:  {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_val_pred):.4f}")
print(f"Recall:    {recall_score(y_val, y_val_pred):.4f}")
print(f"F1 Score:  {f1_score(y_val, y_val_pred):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_val, y_val_proba):.4f}")

# Store baseline metrics for monitoring
baseline_metrics = {
    'accuracy': accuracy_score(y_val, y_val_pred),
    'precision': precision_score(y_val, y_val_pred),
    'recall': recall_score(y_val, y_val_pred),
    'f1': f1_score(y_val, y_val_pred),
    'roc_auc': roc_auc_score(y_val, y_val_proba)
}

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📊 Feature Importance:")
print(feature_importance.to_string(index=False))

## 4. Implement Data Drift Detection

Data drift (also called covariate shift) occurs when the distribution of input features changes between training and production.

### Statistical Tests for Drift Detection:
1. **Kolmogorov-Smirnov (KS) Test**: Non-parametric test comparing two distributions
2. **Population Stability Index (PSI)**: Common in finance/credit risk
3. **Jensen-Shannon Divergence**: Symmetric measure of distribution similarity
4. **Chi-Square Test**: For categorical features

In [ ]:
class DataDriftDetector:
    """
    Comprehensive data drift detection for production ML systems.
    
    Implements multiple statistical tests to detect distribution shifts
    in input features between reference (training) and production data.
    """
    
    def __init__(self, reference_data: pd.DataFrame, 
                 categorical_features: List[str] = None):
        """
        Initialize with reference (training) data.
        
        Args:
            reference_data: Training/reference DataFrame
            categorical_features: List of categorical column names
        """
        self.reference_data = reference_data
        self.categorical_features = categorical_features or []
        self.numerical_features = [col for col in reference_data.columns 
                                   if col not in self.categorical_features]
        
        # Pre-compute reference statistics
        self._compute_reference_stats()
    
    def _compute_reference_stats(self):
        """Pre-compute reference distribution statistics for efficiency."""
        self.reference_stats = {}
        
        for feature in self.numerical_features:
            data = self.reference_data[feature].dropna()
            self.reference_stats[feature] = {
                'mean': data.mean(),
                'std': data.std(),
                'min': data.min(),
                'max': data.max(),
                'percentiles': np.percentile(data, [10, 25, 50, 75, 90])
            }
        
        for feature in self.categorical_features:
            self.reference_stats[feature] = {
                'value_counts': self.reference_data[feature].value_counts(normalize=True)
            }
    
    def ks_test(self, production_data: pd.DataFrame, 
                alpha: float = 0.05) -> Dict[str, Dict]:
        """
        Kolmogorov-Smirnov test for numerical features.
        
        The KS test compares the empirical CDFs of two samples.
        - H0: Both samples come from the same distribution
        - If p-value < alpha, reject H0 (drift detected)
        
        Args:
            production_data: Production DataFrame
            alpha: Significance level
            
        Returns:
            Dictionary with test results per feature
        """
        results = {}
        
        for feature in self.numerical_features:
            ref_data = self.reference_data[feature].dropna()
            prod_data = production_data[feature].dropna()
            
            statistic, p_value = ks_2samp(ref_data, prod_data)
            
            results[feature] = {
                'test': 'Kolmogorov-Smirnov',
                'statistic': statistic,
                'p_value': p_value,
                'drift_detected': p_value < alpha,
                'severity': self._classify_drift_severity(statistic, 'ks')
            }
        
        return results
    
    def calculate_psi(self, production_data: pd.DataFrame, 
                      n_bins: int = 10) -> Dict[str, Dict]:
        """
        Population Stability Index (PSI) for numerical features.
        
        PSI = Σ (Actual% - Expected%) × ln(Actual% / Expected%)
        
        Interpretation:
        - PSI < 0.1: No significant change
        - 0.1 ≤ PSI < 0.25: Moderate change, investigation needed
        - PSI ≥ 0.25: Significant change, action required
        
        Args:
            production_data: Production DataFrame
            n_bins: Number of bins for discretization
            
        Returns:
            Dictionary with PSI values per feature
        """
        results = {}
        
        for feature in self.numerical_features:
            ref_data = self.reference_data[feature].dropna()
            prod_data = production_data[feature].dropna()
            
            # Create bins based on reference data
            bins = np.percentile(ref_data, np.linspace(0, 100, n_bins + 1))
            bins[0] = -np.inf
            bins[-1] = np.inf
            
            # Calculate proportions in each bin
            ref_counts, _ = np.histogram(ref_data, bins=bins)
            prod_counts, _ = np.histogram(prod_data, bins=bins)
            
            # Convert to proportions (add small epsilon to avoid division by zero)
            epsilon = 1e-10
            ref_props = (ref_counts + epsilon) / (ref_counts.sum() + n_bins * epsilon)
            prod_props = (prod_counts + epsilon) / (prod_counts.sum() + n_bins * epsilon)
            
            # Calculate PSI
            psi = np.sum((prod_props - ref_props) * np.log(prod_props / ref_props))
            
            results[feature] = {
                'test': 'Population Stability Index',
                'psi': psi,
                'drift_detected': psi >= 0.1,
                'severity': self._classify_drift_severity(psi, 'psi'),
                'interpretation': self._interpret_psi(psi)
            }
        
        return results
    
    def jensen_shannon_divergence(self, production_data: pd.DataFrame,
                                   n_bins: int = 50) -> Dict[str, Dict]:
        """
        Jensen-Shannon Divergence for numerical features.
        
        JSD is a symmetric, bounded (0 to 1) measure of similarity
        between two probability distributions.
        
        Args:
            production_data: Production DataFrame
            n_bins: Number of bins for discretization
            
        Returns:
            Dictionary with JSD values per feature
        """
        results = {}
        
        for feature in self.numerical_features:
            ref_data = self.reference_data[feature].dropna()
            prod_data = production_data[feature].dropna()
            
            # Create common bin edges
            all_data = np.concatenate([ref_data, prod_data])
            bins = np.linspace(all_data.min(), all_data.max(), n_bins + 1)
            
            # Calculate histograms
            ref_hist, _ = np.histogram(ref_data, bins=bins, density=True)
            prod_hist, _ = np.histogram(prod_data, bins=bins, density=True)
            
            # Normalize to probability distributions
            ref_hist = ref_hist / (ref_hist.sum() + 1e-10)
            prod_hist = prod_hist / (prod_hist.sum() + 1e-10)
            
            # Calculate Jensen-Shannon divergence
            jsd = jensenshannon(ref_hist, prod_hist) ** 2  # Square to get divergence
            
            results[feature] = {
                'test': 'Jensen-Shannon Divergence',
                'jsd': jsd,
                'drift_detected': jsd > 0.1,
                'severity': self._classify_drift_severity(jsd, 'jsd')
            }
        
        return results
    
    def chi_square_test(self, production_data: pd.DataFrame,
                        alpha: float = 0.05) -> Dict[str, Dict]:
        """
        Chi-Square test for categorical features.
        
        Tests if the distribution of categories has changed significantly.
        
        Args:
            production_data: Production DataFrame
            alpha: Significance level
            
        Returns:
            Dictionary with test results per categorical feature
        """
        results = {}
        
        for feature in self.categorical_features:
            # Get category counts
            ref_counts = self.reference_data[feature].value_counts()
            prod_counts = production_data[feature].value_counts()
            
            # Align categories
            all_categories = set(ref_counts.index) | set(prod_counts.index)
            ref_counts = ref_counts.reindex(all_categories, fill_value=0)
            prod_counts = prod_counts.reindex(all_categories, fill_value=0)
            
            # Create contingency table
            contingency = np.array([ref_counts.values, prod_counts.values])
            
            # Chi-square test
            chi2, p_value, dof, expected = chi2_contingency(contingency)
            
            results[feature] = {
                'test': 'Chi-Square',
                'statistic': chi2,
                'p_value': p_value,
                'degrees_of_freedom': dof,
                'drift_detected': p_value < alpha
            }
        
        return results
    
    def _classify_drift_severity(self, value: float, test_type: str) -> str:
        """Classify drift severity based on test type and value."""
        if test_type == 'ks':
            if value < 0.1:
                return 'none'
            elif value < 0.2:
                return 'low'
            elif value < 0.3:
                return 'medium'
            else:
                return 'high'
        elif test_type == 'psi':
            if value < 0.1:
                return 'none'
            elif value < 0.25:
                return 'medium'
            else:
                return 'high'
        elif test_type == 'jsd':
            if value < 0.05:
                return 'none'
            elif value < 0.1:
                return 'low'
            elif value < 0.2:
                return 'medium'
            else:
                return 'high'
        return 'unknown'
    
    def _interpret_psi(self, psi: float) -> str:
        """Provide human-readable interpretation of PSI value."""
        if psi < 0.1:
            return "No significant change - model is stable"
        elif psi < 0.25:
            return "Moderate change - investigate and monitor closely"
        else:
            return "Significant change - action required (retrain/review)"
    
    def comprehensive_drift_report(self, production_data: pd.DataFrame) -> pd.DataFrame:
        """
        Generate a comprehensive drift report using all tests.
        
        Args:
            production_data: Production DataFrame
            
        Returns:
            DataFrame with drift analysis for all features
        """
        ks_results = self.ks_test(production_data)
        psi_results = self.calculate_psi(production_data)
        jsd_results = self.jensen_shannon_divergence(production_data)
        
        report_data = []
        
        for feature in self.numerical_features:
            report_data.append({
                'feature': feature,
                'ks_statistic': ks_results[feature]['statistic'],
                'ks_p_value': ks_results[feature]['p_value'],
                'ks_drift': ks_results[feature]['drift_detected'],
                'psi': psi_results[feature]['psi'],
                'psi_drift': psi_results[feature]['drift_detected'],
                'jsd': jsd_results[feature]['jsd'],
                'jsd_drift': jsd_results[feature]['drift_detected'],
                'overall_drift': (ks_results[feature]['drift_detected'] or 
                                  psi_results[feature]['drift_detected'])
            })
        
        return pd.DataFrame(report_data)

print("✅ DataDriftDetector class created!")

In [ ]:
# Initialize drift detector with training data
drift_detector = DataDriftDetector(
    reference_data=X_train,
    categorical_features=['market_cap']
)

# Test drift detection on different scenarios
print("=" * 70)
print("DATA DRIFT DETECTION RESULTS")
print("=" * 70)

# Scenario 1: No Drift
print("\n📊 SCENARIO 1: No Drift (Production = Normal Regime)")
print("-" * 50)
report_no_drift = drift_detector.comprehensive_drift_report(X_prod_no_drift)
print(report_no_drift.to_string(index=False))
n_drift_features = report_no_drift['overall_drift'].sum()
print(f"\n✅ Features with drift detected: {n_drift_features}/{len(report_no_drift)}")

# Scenario 2: Data Drift
print("\n\n📊 SCENARIO 2: Data Drift (High Volatility Regime)")
print("-" * 50)
report_data_drift = drift_detector.comprehensive_drift_report(X_prod_data_drift)
print(report_data_drift.to_string(index=False))
n_drift_features = report_data_drift['overall_drift'].sum()
print(f"\n⚠️ Features with drift detected: {n_drift_features}/{len(report_data_drift)}")

In [ ]:
# Visualize PSI values across scenarios
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PSI Comparison
scenarios = ['No Drift', 'Data Drift']
colors = ['green', 'red']

for idx, (scenario, report, color) in enumerate([
    ('No Drift', report_no_drift, 'green'),
    ('Data Drift', report_data_drift, 'red')
]):
    ax = axes[idx]
    features = report['feature']
    psi_values = report['psi']
    
    bars = ax.barh(features, psi_values, color=color, alpha=0.7)
    
    # Add threshold lines
    ax.axvline(x=0.1, color='orange', linestyle='--', linewidth=2, label='Moderate (0.1)')
    ax.axvline(x=0.25, color='red', linestyle='--', linewidth=2, label='High (0.25)')
    
    ax.set_xlabel('PSI Value')
    ax.set_title(f'{scenario}: Population Stability Index', fontweight='bold')
    ax.legend(loc='lower right')
    ax.set_xlim(0, max(0.5, psi_values.max() * 1.1))

plt.tight_layout()
plt.show()

print("\n📈 PSI Interpretation:")
print("   - PSI < 0.10: No significant change (Green zone)")
print("   - 0.10 ≤ PSI < 0.25: Moderate change, monitor closely (Orange zone)")
print("   - PSI ≥ 0.25: Significant change, action required (Red zone)")

## 5. Implement Prediction Drift Detection

Prediction drift monitors changes in the model's output distribution over time, regardless of whether input features have shifted. This can indicate:
- Model degradation
- Changes in the target population
- Upstream data issues

In [ ]:
class PredictionDriftDetector:
    """
    Monitor drift in model predictions over time.
    
    Tracks:
    - Prediction probability distribution
    - Class distribution (for classifiers)
    - Prediction confidence levels
    """
    
    def __init__(self, reference_predictions: np.ndarray,
                 reference_probabilities: np.ndarray = None):
        """
        Initialize with reference predictions from validation set.
        
        Args:
            reference_predictions: Class predictions from validation
            reference_probabilities: Probability predictions (if available)
        """
        self.reference_predictions = reference_predictions
        self.reference_probabilities = reference_probabilities
        
        # Store reference statistics
        self.ref_class_distribution = np.bincount(reference_predictions) / len(reference_predictions)
        
        if reference_probabilities is not None:
            self.ref_prob_mean = np.mean(reference_probabilities)
            self.ref_prob_std = np.std(reference_probabilities)
    
    def detect_prediction_drift(self, production_predictions: np.ndarray,
                                 production_probabilities: np.ndarray = None,
                                 alpha: float = 0.05) -> Dict:
        """
        Detect drift in predictions using statistical tests.
        
        Args:
            production_predictions: New predictions to compare
            production_probabilities: New probability predictions
            alpha: Significance level
            
        Returns:
            Dictionary with drift detection results
        """
        results = {}
        
        # 1. Chi-Square test for class distribution
        ref_counts = np.bincount(self.reference_predictions, 
                                  minlength=max(self.reference_predictions.max(),
                                               production_predictions.max()) + 1)
        prod_counts = np.bincount(production_predictions,
                                   minlength=max(self.reference_predictions.max(),
                                                production_predictions.max()) + 1)
        
        # Normalize to same scale
        ref_counts_scaled = ref_counts * len(production_predictions) / len(self.reference_predictions)
        
        # Chi-square test
        chi2_stat = np.sum((prod_counts - ref_counts_scaled) ** 2 / (ref_counts_scaled + 1e-10))
        p_value = 1 - stats.chi2.cdf(chi2_stat, df=len(ref_counts) - 1)
        
        results['class_distribution'] = {
            'chi2_statistic': chi2_stat,
            'p_value': p_value,
            'drift_detected': p_value < alpha,
            'reference_distribution': self.ref_class_distribution,
            'production_distribution': prod_counts / prod_counts.sum()
        }
        
        # 2. KS test for probability distribution (if available)
        if production_probabilities is not None and self.reference_probabilities is not None:
            ks_stat, ks_p_value = ks_2samp(self.reference_probabilities, 
                                           production_probabilities)
            
            results['probability_distribution'] = {
                'ks_statistic': ks_stat,
                'p_value': ks_p_value,
                'drift_detected': ks_p_value < alpha,
                'reference_mean': self.ref_prob_mean,
                'production_mean': np.mean(production_probabilities),
                'mean_shift': np.mean(production_probabilities) - self.ref_prob_mean
            }
        
        # 3. Confidence drift (are predictions becoming less confident?)
        if production_probabilities is not None and self.reference_probabilities is not None:
            # Confidence = distance from 0.5 decision boundary
            ref_confidence = np.abs(self.reference_probabilities - 0.5)
            prod_confidence = np.abs(production_probabilities - 0.5)
            
            conf_stat, conf_p_value = ks_2samp(ref_confidence, prod_confidence)
            
            results['confidence'] = {
                'ks_statistic': conf_stat,
                'p_value': conf_p_value,
                'drift_detected': conf_p_value < alpha,
                'reference_mean_confidence': np.mean(ref_confidence),
                'production_mean_confidence': np.mean(prod_confidence)
            }
        
        return results
    
    def calculate_psi_predictions(self, production_probabilities: np.ndarray,
                                   n_bins: int = 10) -> float:
        """Calculate PSI for probability predictions."""
        if self.reference_probabilities is None:
            return None
        
        bins = np.linspace(0, 1, n_bins + 1)
        
        ref_counts, _ = np.histogram(self.reference_probabilities, bins=bins)
        prod_counts, _ = np.histogram(production_probabilities, bins=bins)
        
        epsilon = 1e-10
        ref_props = (ref_counts + epsilon) / (ref_counts.sum() + n_bins * epsilon)
        prod_props = (prod_counts + epsilon) / (prod_counts.sum() + n_bins * epsilon)
        
        psi = np.sum((prod_props - ref_props) * np.log(prod_props / ref_props))
        
        return psi

print("✅ PredictionDriftDetector class created!")

In [ ]:
# Generate predictions for all scenarios
def get_predictions(X, model, scaler):
    """Scale data and get model predictions."""
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    probabilities = model.predict_proba(X_scaled)[:, 1]
    return predictions, probabilities

# Reference predictions (validation set)
y_val_pred, y_val_proba = get_predictions(X_val, model, scaler)

# Production predictions
pred_no_drift, proba_no_drift = get_predictions(X_prod_no_drift, model, scaler)
pred_data_drift, proba_data_drift = get_predictions(X_prod_data_drift, model, scaler)
pred_concept_drift, proba_concept_drift = get_predictions(X_prod_concept_drift, model, scaler)
pred_both_drift, proba_both_drift = get_predictions(X_prod_both_drift, model, scaler)

# Initialize prediction drift detector
pred_drift_detector = PredictionDriftDetector(
    reference_predictions=y_val_pred,
    reference_probabilities=y_val_proba
)

print("=" * 70)
print("PREDICTION DRIFT DETECTION RESULTS")
print("=" * 70)

scenarios = [
    ('No Drift', pred_no_drift, proba_no_drift),
    ('Data Drift', pred_data_drift, proba_data_drift),
    ('Concept Drift', pred_concept_drift, proba_concept_drift),
    ('Both Drifts', pred_both_drift, proba_both_drift)
]

for name, preds, probas in scenarios:
    print(f"\n📊 {name}:")
    results = pred_drift_detector.detect_prediction_drift(preds, probas)
    
    # Class distribution
    class_drift = results['class_distribution']
    print(f"   Class Distribution Drift: {'⚠️ YES' if class_drift['drift_detected'] else '✅ NO'}")
    print(f"   - Chi² p-value: {class_drift['p_value']:.4f}")
    
    # Probability distribution
    prob_drift = results['probability_distribution']
    print(f"   Probability Distribution Drift: {'⚠️ YES' if prob_drift['drift_detected'] else '✅ NO'}")
    print(f"   - KS p-value: {prob_drift['p_value']:.4f}")
    print(f"   - Mean shift: {prob_drift['mean_shift']:+.4f}")
    
    # PSI
    psi = pred_drift_detector.calculate_psi_predictions(probas)
    print(f"   Prediction PSI: {psi:.4f}")

In [ ]:
# Visualize prediction probability distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

scenarios_data = [
    ('No Drift', proba_no_drift, 'green'),
    ('Data Drift', proba_data_drift, 'orange'),
    ('Concept Drift', proba_concept_drift, 'red'),
    ('Both Drifts', proba_both_drift, 'darkred')
]

for idx, (name, probas, color) in enumerate(scenarios_data):
    ax = axes[idx // 2, idx % 2]
    
    # Reference distribution
    ax.hist(y_val_proba, bins=30, alpha=0.5, label='Reference (Validation)', 
            density=True, color='blue')
    # Production distribution
    ax.hist(probas, bins=30, alpha=0.5, label=f'Production ({name})',
            density=True, color=color)
    
    ax.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Decision Boundary')
    ax.set_xlabel('Predicted Probability')
    ax.set_ylabel('Density')
    ax.set_title(f'{name}: Prediction Distribution', fontweight='bold')
    ax.legend()
    ax.set_xlim(0, 1)

plt.suptitle('Prediction Drift: Reference vs Production Distributions', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Implement Concept Drift Detection

Concept drift occurs when the relationship between features and the target variable changes. This is the most insidious type of drift because:
- Input features may look normal (no data drift)
- Predictions may look normal (no prediction drift)
- But the model is making wrong predictions!

### Detection Methods:
1. **Performance Monitoring**: Track accuracy, precision, recall over time windows
2. **ADWIN (Adaptive Windowing)**: Detects distribution changes in a stream
3. **Page-Hinkley Test**: Sequential change detection
4. **Error Rate Monitoring**: Track prediction errors over time

In [ ]:
class ConceptDriftDetector:
    """
    Detect concept drift using performance monitoring and statistical tests.
    
    Concept drift = the relationship P(Y|X) changes over time.
    """
    
    def __init__(self, baseline_metrics: Dict[str, float],
                 window_size: int = 100,
                 significance_threshold: float = 0.05):
        """
        Initialize with baseline performance metrics.
        
        Args:
            baseline_metrics: Dictionary of baseline metrics (accuracy, f1, etc.)
            window_size: Size of rolling window for monitoring
            significance_threshold: Threshold for statistical significance
        """
        self.baseline_metrics = baseline_metrics
        self.window_size = window_size
        self.significance_threshold = significance_threshold
        
        # Store historical metrics
        self.metric_history = {metric: [] for metric in baseline_metrics.keys()}
        self.alert_history = []
    
    def update(self, y_true: np.ndarray, y_pred: np.ndarray, 
               y_proba: np.ndarray = None, timestamp: datetime = None):
        """
        Update with new batch of predictions and compute metrics.
        
        Args:
            y_true: True labels
            y_pred: Predicted labels
            y_proba: Predicted probabilities (optional)
            timestamp: Timestamp for this batch
        """
        timestamp = timestamp or datetime.now()
        
        # Compute metrics
        current_metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0)
        }
        
        if y_proba is not None:
            try:
                current_metrics['roc_auc'] = roc_auc_score(y_true, y_proba)
            except:
                current_metrics['roc_auc'] = 0.5
        
        # Store metrics
        for metric, value in current_metrics.items():
            if metric in self.metric_history:
                self.metric_history[metric].append({
                    'timestamp': timestamp,
                    'value': value
                })
        
        return current_metrics
    
    def detect_drift(self, current_metrics: Dict[str, float],
                     method: str = 'threshold') -> Dict:
        """
        Detect concept drift based on performance degradation.
        
        Args:
            current_metrics: Current batch metrics
            method: Detection method ('threshold', 'statistical', 'adwin')
            
        Returns:
            Dictionary with drift detection results
        """
        results = {}
        
        for metric, current_value in current_metrics.items():
            if metric not in self.baseline_metrics:
                continue
            
            baseline_value = self.baseline_metrics[metric]
            
            if method == 'threshold':
                # Simple threshold-based detection
                # Alert if metric drops more than 10% from baseline
                relative_drop = (baseline_value - current_value) / (baseline_value + 1e-10)
                drift_detected = relative_drop > 0.10
                
                results[metric] = {
                    'baseline': baseline_value,
                    'current': current_value,
                    'relative_drop': relative_drop,
                    'drift_detected': drift_detected,
                    'severity': self._classify_severity(relative_drop)
                }
            
            elif method == 'statistical':
                # Use historical data for statistical test
                if len(self.metric_history.get(metric, [])) >= self.window_size:
                    recent_values = [h['value'] for h in self.metric_history[metric][-self.window_size:]]
                    
                    # One-sample t-test against baseline
                    t_stat, p_value = stats.ttest_1samp(recent_values, baseline_value)
                    
                    # Check if significantly lower (one-tailed)
                    drift_detected = (p_value / 2 < self.significance_threshold and 
                                     np.mean(recent_values) < baseline_value)
                    
                    results[metric] = {
                        'baseline': baseline_value,
                        'current_mean': np.mean(recent_values),
                        't_statistic': t_stat,
                        'p_value': p_value,
                        'drift_detected': drift_detected
                    }
        
        return results
    
    def page_hinkley_test(self, values: List[float], 
                          delta: float = 0.005,
                          threshold: float = 50) -> Tuple[bool, int]:
        """
        Page-Hinkley test for change detection in a sequence.
        
        Detects a change in the mean of a Gaussian signal.
        
        Args:
            values: Sequence of values to test
            delta: Minimum amplitude of change to detect
            threshold: Decision threshold
            
        Returns:
            Tuple of (change_detected, change_point_index)
        """
        n = len(values)
        if n < 10:
            return False, -1
        
        mean = np.mean(values)
        
        # Cumulative sum statistics
        m_t = 0
        M_t = 0
        
        for t in range(n):
            m_t += values[t] - mean - delta
            M_t = max(M_t, m_t)
            
            ph_t = M_t - m_t
            
            if ph_t > threshold:
                return True, t
        
        return False, -1
    
    def _classify_severity(self, relative_drop: float) -> str:
        """Classify drift severity based on relative performance drop."""
        if relative_drop < 0.05:
            return 'none'
        elif relative_drop < 0.10:
            return 'low'
        elif relative_drop < 0.20:
            return 'medium'
        else:
            return 'high'
    
    def get_performance_trend(self, metric: str = 'accuracy') -> pd.DataFrame:
        """Get historical trend for a specific metric."""
        if metric not in self.metric_history:
            return pd.DataFrame()
        
        return pd.DataFrame(self.metric_history[metric])

print("✅ ConceptDriftDetector class created!")

In [ ]:
# Evaluate model performance across all scenarios
print("=" * 70)
print("CONCEPT DRIFT DETECTION: PERFORMANCE COMPARISON")
print("=" * 70)

# Initialize concept drift detector
concept_detector = ConceptDriftDetector(baseline_metrics)

scenarios_full = [
    ('No Drift', X_prod_no_drift, y_prod_no_drift),
    ('Data Drift Only', X_prod_data_drift, y_prod_data_drift),
    ('Concept Drift Only', X_prod_concept_drift, y_prod_concept_drift),
    ('Both Drifts', X_prod_both_drift, y_prod_both_drift)
]

performance_results = []

for name, X_prod, y_true in scenarios_full:
    # Get predictions
    X_scaled = scaler.transform(X_prod)
    y_pred = model.predict(X_scaled)
    y_proba = model.predict_proba(X_scaled)[:, 1]
    
    # Compute metrics
    metrics = {
        'scenario': name,
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_proba) if len(np.unique(y_true)) > 1 else 0.5
    }
    
    performance_results.append(metrics)
    
    # Detect drift
    current_metrics = {k: v for k, v in metrics.items() if k != 'scenario'}
    drift_results = concept_detector.detect_drift(current_metrics)
    
    print(f"\n📊 {name}:")
    for metric, result in drift_results.items():
        status = '🔴' if result['drift_detected'] else '🟢'
        drop = result['relative_drop']
        print(f"   {metric:12}: {result['current']:.4f} (baseline: {result['baseline']:.4f}, "
              f"drop: {drop:+.1%}) {status}")

# Create comparison DataFrame
perf_df = pd.DataFrame(performance_results)
print("\n\n📈 Performance Summary Table:")
print(perf_df.to_string(index=False))

In [ ]:
# Simulate streaming data with gradual concept drift
print("=" * 70)
print("SIMULATING STREAMING DATA WITH GRADUAL DRIFT")
print("=" * 70)

# Generate time series of batches with gradual drift
n_batches = 30
batch_size = 200
drift_start_batch = 15  # Drift starts at batch 15

streaming_metrics = []

for batch_idx in range(n_batches):
    # Calculate drift intensity (0 before drift_start, increases after)
    if batch_idx < drift_start_batch:
        drift_factor = 0.0
        regime = 'normal'
        label_regime = 'normal'
    else:
        # Gradual increase in drift
        drift_factor = (batch_idx - drift_start_batch) / (n_batches - drift_start_batch)
        regime = 'gradual'
        label_regime = 'concept_drift' if drift_factor > 0.3 else 'normal'
    
    # Generate batch data
    X_batch = generate_financial_features(batch_size, regime=regime, drift_factor=drift_factor)
    y_batch = generate_labels(X_batch, regime=label_regime, noise_level=0.1 + drift_factor * 0.1)
    
    # Get predictions
    X_batch_scaled = scaler.transform(X_batch)
    y_pred_batch = model.predict(X_batch_scaled)
    y_proba_batch = model.predict_proba(X_batch_scaled)[:, 1]
    
    # Compute metrics
    batch_metrics = {
        'batch': batch_idx,
        'drift_factor': drift_factor,
        'accuracy': accuracy_score(y_batch, y_pred_batch),
        'f1': f1_score(y_batch, y_pred_batch, zero_division=0),
        'roc_auc': roc_auc_score(y_batch, y_proba_batch) if len(np.unique(y_batch)) > 1 else 0.5
    }
    streaming_metrics.append(batch_metrics)

streaming_df = pd.DataFrame(streaming_metrics)

# Visualize performance over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Accuracy over time
ax1 = axes[0, 0]
ax1.plot(streaming_df['batch'], streaming_df['accuracy'], 'b-', linewidth=2, marker='o', markersize=4)
ax1.axhline(y=baseline_metrics['accuracy'], color='green', linestyle='--', label='Baseline')
ax1.axvline(x=drift_start_batch, color='red', linestyle='--', alpha=0.7, label='Drift Start')
ax1.fill_between(streaming_df['batch'], 
                  baseline_metrics['accuracy'] * 0.9, baseline_metrics['accuracy'] * 1.1,
                  alpha=0.2, color='green', label='±10% Threshold')
ax1.set_xlabel('Batch')
ax1.set_ylabel('Accuracy')
ax1.set_title('Accuracy Over Time', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# F1 Score over time
ax2 = axes[0, 1]
ax2.plot(streaming_df['batch'], streaming_df['f1'], 'b-', linewidth=2, marker='o', markersize=4)
ax2.axhline(y=baseline_metrics['f1'], color='green', linestyle='--', label='Baseline')
ax2.axvline(x=drift_start_batch, color='red', linestyle='--', alpha=0.7, label='Drift Start')
ax2.set_xlabel('Batch')
ax2.set_ylabel('F1 Score')
ax2.set_title('F1 Score Over Time', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# ROC-AUC over time
ax3 = axes[1, 0]
ax3.plot(streaming_df['batch'], streaming_df['roc_auc'], 'b-', linewidth=2, marker='o', markersize=4)
ax3.axhline(y=baseline_metrics['roc_auc'], color='green', linestyle='--', label='Baseline')
ax3.axvline(x=drift_start_batch, color='red', linestyle='--', alpha=0.7, label='Drift Start')
ax3.set_xlabel('Batch')
ax3.set_ylabel('ROC-AUC')
ax3.set_title('ROC-AUC Over Time', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Drift factor vs Performance
ax4 = axes[1, 1]
ax4.scatter(streaming_df['drift_factor'], streaming_df['accuracy'], c=streaming_df['batch'], 
            cmap='viridis', s=100, alpha=0.7)
ax4.set_xlabel('Drift Factor')
ax4.set_ylabel('Accuracy')
ax4.set_title('Accuracy vs Drift Intensity', fontweight='bold')
cbar = plt.colorbar(ax4.collections[0], ax=ax4)
cbar.set_label('Batch Number')
ax4.grid(True, alpha=0.3)

plt.suptitle('Performance Degradation with Gradual Concept Drift', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n📉 Key Observations:")
print(f"   - Baseline Accuracy: {baseline_metrics['accuracy']:.4f}")
print(f"   - Final Accuracy: {streaming_df['accuracy'].iloc[-1]:.4f}")
print(f"   - Performance Drop: {baseline_metrics['accuracy'] - streaming_df['accuracy'].iloc[-1]:.4f}")

## 7. Build Performance Monitoring Dashboard

A comprehensive monitoring dashboard tracks multiple metrics and visualizes them in real-time. Here we create a reusable dashboard class.

In [ ]:
class ModelMonitoringDashboard:
    """
    Comprehensive monitoring dashboard for production ML models.
    
    Tracks and visualizes:
    - Model performance metrics over time
    - Data drift indicators
    - Prediction drift
    - Feature distributions
    - Alerts and anomalies
    """
    
    def __init__(self, model, scaler, baseline_metrics: Dict,
                 reference_data: pd.DataFrame,
                 feature_names: List[str]):
        """
        Initialize monitoring dashboard.
        
        Args:
            model: Trained model
            scaler: Feature scaler
            baseline_metrics: Baseline performance metrics
            reference_data: Reference (training) data
            feature_names: List of feature names
        """
        self.model = model
        self.scaler = scaler
        self.baseline_metrics = baseline_metrics
        self.reference_data = reference_data
        self.feature_names = feature_names
        
        # Initialize detectors
        self.data_drift_detector = DataDriftDetector(reference_data)
        
        # Storage for historical data
        self.history = {
            'timestamps': [],
            'metrics': [],
            'data_drift': [],
            'prediction_drift': [],
            'alerts': []
        }
    
    def log_batch(self, X: pd.DataFrame, y_true: np.ndarray,
                  timestamp: datetime = None) -> Dict:
        """
        Log a new batch of predictions and compute all monitoring metrics.
        
        Args:
            X: Feature DataFrame
            y_true: True labels
            timestamp: Batch timestamp
            
        Returns:
            Dictionary with all monitoring results
        """
        timestamp = timestamp or datetime.now()
        
        # Get predictions
        X_scaled = self.scaler.transform(X)
        y_pred = self.model.predict(X_scaled)
        y_proba = self.model.predict_proba(X_scaled)[:, 1]
        
        # 1. Compute performance metrics
        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, zero_division=0),
            'recall': recall_score(y_true, y_pred, zero_division=0),
            'f1': f1_score(y_true, y_pred, zero_division=0),
            'roc_auc': roc_auc_score(y_true, y_proba) if len(np.unique(y_true)) > 1 else 0.5
        }
        
        # 2. Data drift detection
        data_drift = self.data_drift_detector.comprehensive_drift_report(X)
        n_drifted_features = data_drift['overall_drift'].sum()
        
        # 3. Prediction drift (compare to first batch or reference)
        pred_drift_psi = self._calculate_prediction_psi(y_proba)
        
        # 4. Generate alerts
        alerts = self._check_alerts(metrics, n_drifted_features, pred_drift_psi)
        
        # Store in history
        self.history['timestamps'].append(timestamp)
        self.history['metrics'].append(metrics)
        self.history['data_drift'].append({
            'n_drifted': n_drifted_features,
            'total_features': len(self.feature_names)
        })
        self.history['prediction_drift'].append({'psi': pred_drift_psi})
        self.history['alerts'].append(alerts)
        
        return {
            'timestamp': timestamp,
            'metrics': metrics,
            'data_drift': {'n_drifted': n_drifted_features},
            'prediction_drift_psi': pred_drift_psi,
            'alerts': alerts
        }
    
    def _calculate_prediction_psi(self, y_proba: np.ndarray, n_bins: int = 10) -> float:
        """Calculate PSI for prediction probabilities."""
        if len(self.history['timestamps']) == 0:
            self._reference_proba = y_proba
            return 0.0
        
        bins = np.linspace(0, 1, n_bins + 1)
        
        ref_counts, _ = np.histogram(self._reference_proba, bins=bins)
        prod_counts, _ = np.histogram(y_proba, bins=bins)
        
        epsilon = 1e-10
        ref_props = (ref_counts + epsilon) / (ref_counts.sum() + n_bins * epsilon)
        prod_props = (prod_counts + epsilon) / (prod_counts.sum() + n_bins * epsilon)
        
        psi = np.sum((prod_props - ref_props) * np.log(prod_props / ref_props))
        
        return psi
    
    def _check_alerts(self, metrics: Dict, n_drifted: int, 
                      pred_psi: float) -> List[Dict]:
        """Check for alert conditions."""
        alerts = []
        
        # Performance degradation alerts
        for metric, value in metrics.items():
            if metric in self.baseline_metrics:
                baseline = self.baseline_metrics[metric]
                relative_drop = (baseline - value) / (baseline + 1e-10)
                
                if relative_drop > 0.20:
                    alerts.append({
                        'type': 'CRITICAL',
                        'metric': metric,
                        'message': f'{metric} dropped {relative_drop:.1%} from baseline'
                    })
                elif relative_drop > 0.10:
                    alerts.append({
                        'type': 'WARNING',
                        'metric': metric,
                        'message': f'{metric} dropped {relative_drop:.1%} from baseline'
                    })
        
        # Data drift alert
        if n_drifted >= len(self.feature_names) // 2:
            alerts.append({
                'type': 'WARNING',
                'metric': 'data_drift',
                'message': f'{n_drifted}/{len(self.feature_names)} features show drift'
            })
        
        # Prediction drift alert
        if pred_psi > 0.25:
            alerts.append({
                'type': 'CRITICAL',
                'metric': 'prediction_drift',
                'message': f'Prediction PSI = {pred_psi:.3f} (>0.25)'
            })
        elif pred_psi > 0.10:
            alerts.append({
                'type': 'WARNING',
                'metric': 'prediction_drift',
                'message': f'Prediction PSI = {pred_psi:.3f} (>0.10)'
            })
        
        return alerts
    
    def plot_dashboard(self, figsize: Tuple[int, int] = (16, 12)):
        """Generate comprehensive monitoring dashboard visualization."""
        if len(self.history['timestamps']) < 2:
            print("Not enough data to plot. Log at least 2 batches.")
            return
        
        fig = plt.figure(figsize=figsize)
        
        # Create grid layout
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Prepare data
        timestamps = range(len(self.history['timestamps']))
        metrics_df = pd.DataFrame(self.history['metrics'])
        
        # 1. Accuracy over time (top left)
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.plot(timestamps, metrics_df['accuracy'], 'b-', linewidth=2, marker='o', markersize=4)
        ax1.axhline(y=self.baseline_metrics['accuracy'], color='green', linestyle='--', 
                    label='Baseline', linewidth=2)
        ax1.fill_between(timestamps, 
                         self.baseline_metrics['accuracy'] * 0.9,
                         self.baseline_metrics['accuracy'] * 1.1,
                         alpha=0.2, color='green')
        ax1.set_xlabel('Batch')
        ax1.set_ylabel('Accuracy')
        ax1.set_title('Accuracy', fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. F1 Score over time (top middle)
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.plot(timestamps, metrics_df['f1'], 'purple', linewidth=2, marker='o', markersize=4)
        ax2.axhline(y=self.baseline_metrics['f1'], color='green', linestyle='--', linewidth=2)
        ax2.set_xlabel('Batch')
        ax2.set_ylabel('F1 Score')
        ax2.set_title('F1 Score', fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        # 3. ROC-AUC over time (top right)
        ax3 = fig.add_subplot(gs[0, 2])
        ax3.plot(timestamps, metrics_df['roc_auc'], 'orange', linewidth=2, marker='o', markersize=4)
        ax3.axhline(y=self.baseline_metrics['roc_auc'], color='green', linestyle='--', linewidth=2)
        ax3.set_xlabel('Batch')
        ax3.set_ylabel('ROC-AUC')
        ax3.set_title('ROC-AUC', fontweight='bold')
        ax3.grid(True, alpha=0.3)
        
        # 4. Data drift - number of drifted features (middle left)
        ax4 = fig.add_subplot(gs[1, 0])
        n_drifted = [d['n_drifted'] for d in self.history['data_drift']]
        colors = ['green' if n < 2 else 'orange' if n < 4 else 'red' for n in n_drifted]
        ax4.bar(timestamps, n_drifted, color=colors, alpha=0.7)
        ax4.axhline(y=len(self.feature_names) // 2, color='red', linestyle='--', 
                    label='Alert Threshold')
        ax4.set_xlabel('Batch')
        ax4.set_ylabel('# Drifted Features')
        ax4.set_title('Data Drift (# Features)', fontweight='bold')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # 5. Prediction PSI (middle center)
        ax5 = fig.add_subplot(gs[1, 1])
        pred_psi = [d['psi'] for d in self.history['prediction_drift']]
        ax5.plot(timestamps, pred_psi, 'red', linewidth=2, marker='s', markersize=4)
        ax5.axhline(y=0.10, color='orange', linestyle='--', label='Moderate (0.10)')
        ax5.axhline(y=0.25, color='red', linestyle='--', label='High (0.25)')
        ax5.set_xlabel('Batch')
        ax5.set_ylabel('PSI')
        ax5.set_title('Prediction Drift (PSI)', fontweight='bold')
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        
        # 6. Precision & Recall (middle right)
        ax6 = fig.add_subplot(gs[1, 2])
        ax6.plot(timestamps, metrics_df['precision'], 'b-', linewidth=2, marker='o', 
                 markersize=4, label='Precision')
        ax6.plot(timestamps, metrics_df['recall'], 'r-', linewidth=2, marker='s', 
                 markersize=4, label='Recall')
        ax6.set_xlabel('Batch')
        ax6.set_ylabel('Score')
        ax6.set_title('Precision & Recall', fontweight='bold')
        ax6.legend()
        ax6.grid(True, alpha=0.3)
        
        # 7. Alert summary (bottom, spans full width)
        ax7 = fig.add_subplot(gs[2, :])
        
        # Count alerts by type
        alert_counts = {'WARNING': [], 'CRITICAL': []}
        for batch_alerts in self.history['alerts']:
            warning_count = sum(1 for a in batch_alerts if a['type'] == 'WARNING')
            critical_count = sum(1 for a in batch_alerts if a['type'] == 'CRITICAL')
            alert_counts['WARNING'].append(warning_count)
            alert_counts['CRITICAL'].append(critical_count)
        
        width = 0.35
        x = np.array(timestamps)
        ax7.bar(x - width/2, alert_counts['WARNING'], width, label='Warning', color='orange', alpha=0.7)
        ax7.bar(x + width/2, alert_counts['CRITICAL'], width, label='Critical', color='red', alpha=0.7)
        ax7.set_xlabel('Batch')
        ax7.set_ylabel('# Alerts')
        ax7.set_title('Alerts Over Time', fontweight='bold')
        ax7.legend()
        ax7.grid(True, alpha=0.3)
        
        plt.suptitle('Model Monitoring Dashboard', fontsize=16, fontweight='bold', y=0.98)
        plt.show()
    
    def get_summary_report(self) -> str:
        """Generate a text summary report."""
        if len(self.history['timestamps']) == 0:
            return "No data logged yet."
        
        latest = self.history['metrics'][-1]
        n_batches = len(self.history['timestamps'])
        total_alerts = sum(len(a) for a in self.history['alerts'])
        critical_alerts = sum(sum(1 for alert in a if alert['type'] == 'CRITICAL') 
                             for a in self.history['alerts'])
        
        report = f"""
╔══════════════════════════════════════════════════════════════════╗
║                    MODEL MONITORING SUMMARY                       ║
╠══════════════════════════════════════════════════════════════════╣
║  Batches Processed: {n_batches:>5}                                        ║
║  Total Alerts: {total_alerts:>5} (Critical: {critical_alerts})                           ║
╠══════════════════════════════════════════════════════════════════╣
║  LATEST METRICS:                                                  ║
║    Accuracy:  {latest['accuracy']:.4f} (baseline: {self.baseline_metrics['accuracy']:.4f})              ║
║    F1 Score:  {latest['f1']:.4f} (baseline: {self.baseline_metrics['f1']:.4f})              ║
║    ROC-AUC:   {latest['roc_auc']:.4f} (baseline: {self.baseline_metrics['roc_auc']:.4f})              ║
╠══════════════════════════════════════════════════════════════════╣
║  DATA DRIFT: {self.history['data_drift'][-1]['n_drifted']}/{self.history['data_drift'][-1]['total_features']} features drifted                           ║
║  PREDICTION PSI: {self.history['prediction_drift'][-1]['psi']:.4f}                                     ║
╚══════════════════════════════════════════════════════════════════╝
"""
        return report

print("✅ ModelMonitoringDashboard class created!")